In [2]:
import pandas as pd
import numpy as np
import os
import glob
from itertools import combinations
from collections import defaultdict
from random import sample
import random
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('poster')
sns.set_theme(style="whitegrid")
#pandas max row

In [3]:
def read_files(file_list):
    file_list.sort()
    print(f"Total files: {len(file_list)}")
    all_results = []
    for file in file_list:
        df = pd.read_csv(file)
        all_results.append(df)
    return pd.concat(all_results)

In [4]:
gilon_activity_main_dir = 'dataset/importance_result_files_mine/gilon_activity/'
shufflenet_activity = glob.glob(gilon_activity_main_dir + 'shufflenet/*')
resnet_activity = glob.glob(gilon_activity_main_dir + 'resnet9/*')
mlpmixer_activity = glob.glob(gilon_activity_main_dir + 'mlpmixer/*')
simplevit_activity = glob.glob(gilon_activity_main_dir + 'simplevit/*')

shufflenet_activity_df = read_files(shufflenet_activity)
resnet_activity_df = read_files(resnet_activity)
mlpmixer_activity_df = read_files(mlpmixer_activity)
simplevit_activity_df = read_files(simplevit_activity)

shufflenet_gilon_qr_true = shufflenet_activity_df[shufflenet_activity_df['opl_loss']==True]
shufflenet_gilon_qr_false = shufflenet_activity_df[shufflenet_activity_df['opl_loss']==False]
resnet_gilon_qr_true = resnet_activity_df[resnet_activity_df['opl_loss']==True]
resnet_gilon_qr_false = resnet_activity_df[resnet_activity_df['opl_loss']==False]
mlpmixer_gilon_qr_true = mlpmixer_activity_df[mlpmixer_activity_df['opl_loss']==True]
mlpmixer_gilon_qr_false = mlpmixer_activity_df[mlpmixer_activity_df['opl_loss']==False]
simplevit_gilon_qr_true = simplevit_activity_df[simplevit_activity_df['opl_loss']==True]
simplevit_gilon_qr_false = simplevit_activity_df[simplevit_activity_df['opl_loss']==False]

all_results = [[shufflenet_gilon_qr_false, 'ShuffleNet(CE)'],
                [shufflenet_gilon_qr_true, 'ShuffleNet(QR)'],
                [resnet_gilon_qr_false, 'ResNet9(CE)'],
                [resnet_gilon_qr_true, 'ResNet9(QR)'],
                [mlpmixer_gilon_qr_false, 'MLPMixer(CE)'],
                [mlpmixer_gilon_qr_true, 'MLPMixer(QR)'],
                [simplevit_gilon_qr_false, 'SimpleViT(CE)'],
                [simplevit_gilon_qr_true, 'SimpleViT(QR)']]

Total files: 6
Total files: 6
Total files: 6
Total files: 5


In [5]:
def calculate_drop_acc_test(df, drop_k=2):
    df = df[df['exp_type']=='important'].groupby('exp_num').mean()
    base_acc = df[df['feature_order']==0]['accuracy'].mean()
    k_acc = df[df['feature_order']==drop_k]['accuracy'].mean()

    base_loss = df[df['feature_order']==0]['test_loss'].mean()
    k_loss = df[df['feature_order']==drop_k]['test_loss'].mean()
    # calculate as percentage
    drop_acc = (base_acc - k_acc) / base_acc * 100
    drop_loss = (k_loss - base_loss) / base_loss * 100
    print(f"Drop {drop_k} accuracy: {drop_acc:.3f}")
    print(f"Drop {drop_k} loss: {drop_loss:.3f}")

    # calculate drop weighted accuracy
    accuracies = df[df['feature_order']!=0]['accuracy'].values
    dacc_values = [base_acc - acc for acc in accuracies]
    # place more weights on dacc values with less features dropped
    weights_rev = [1/len(dacc_values) * i for i in range(len(dacc_values), 0, -1)]
    weighted_sum = sum([dacc * weight for dacc, weight in zip(dacc_values, weights_rev)])


    return [drop_acc, drop_loss, drop_k, base_acc, k_acc, base_loss, k_loss, weighted_sum]

In [6]:
all_drop_results = []
for result in all_results:
    drop_results = calculate_drop_acc_test(result[0], drop_k=3)
    all_drop_results.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6], drop_results[7]])
all_drop_results_df = pd.DataFrame(all_drop_results, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss', 'weighted_acc_drop',])
all_drop_results_df

Drop 3 accuracy: 3.326
Drop 3 loss: 22.894
Drop 3 accuracy: 8.158
Drop 3 loss: 95.767
Drop 3 accuracy: 1.553
Drop 3 loss: 36.708
Drop 3 accuracy: 6.393
Drop 3 loss: 111.080
Drop 3 accuracy: 8.118
Drop 3 loss: 25.219
Drop 3 accuracy: 8.106
Drop 3 loss: 26.634
Drop 3 accuracy: 3.512
Drop 3 loss: 26.916
Drop 3 accuracy: 2.047
Drop 3 loss: 20.891


/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupb

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss,weighted_acc_drop
0,ShuffleNet(CE),3.325582,22.893906,3,0.957756,0.925905,0.260112,0.319661,0.478584
1,ShuffleNet(QR),8.157610,95.766512,3,0.945206,0.868100,0.288708,0.565194,0.760747
2,ResNet9(CE),1.553384,36.708291,3,0.960291,0.945374,0.189632,0.259242,0.420436
3,ResNet9(QR),6.393184,111.080478,3,0.940318,0.880202,0.241312,0.509363,0.615026
4,MLPMixer(CE),8.117946,25.219011,3,0.919630,0.844975,0.350531,0.438931,0.531163
5,MLPMixer(QR),8.105685,26.633796,3,0.921885,0.847160,0.396319,0.501873,0.697497
6,SimpleViT(CE),3.512102,26.915633,3,0.922446,0.890048,0.248335,0.315176,0.387801
7,SimpleViT(QR),2.046664,20.890970,3,0.923888,0.904979,0.245232,0.296464,0.456485


# MS

In [7]:
microsoft_main_dir = 'dataset/importance_result_files_mine/microsoft_activity_new/'
shufflenet_ms = glob.glob(microsoft_main_dir + 'shufflenet/*')
resnet_ms = glob.glob(microsoft_main_dir + 'resnet9/*')
mlpmixer_ms = glob.glob(microsoft_main_dir + 'mlpmixer/*')
simplevit_ms = glob.glob(microsoft_main_dir + 'simplevit/*')

shufflenet_ms_df = read_files(shufflenet_ms)
resnet_ms_df = read_files(resnet_ms)
mlpmixer_ms_df = read_files(mlpmixer_ms)
simplevit_ms_df = read_files(simplevit_ms)

shufflenet_ms_qr_true = shufflenet_ms_df[shufflenet_ms_df['opl_loss']==True]
shufflenet_ms_qr_false = shufflenet_ms_df[shufflenet_ms_df['opl_loss']==False]
resnet_ms_qr_true = resnet_ms_df[resnet_ms_df['opl_loss']==True]
resnet_ms_qr_false = resnet_ms_df[resnet_ms_df['opl_loss']==False]
mlpmixer_ms_qr_true = mlpmixer_ms_df[mlpmixer_ms_df['opl_loss']==True]
mlpmixer_ms_qr_false = mlpmixer_ms_df[mlpmixer_ms_df['opl_loss']==False]
simplevit_ms_qr_true = simplevit_ms_df[simplevit_ms_df['opl_loss']==True]
simplevit_ms_qr_false = simplevit_ms_df[simplevit_ms_df['opl_loss']==False]

all_results_ms = [[shufflenet_ms_qr_false, 'ShuffleNet(CE)'],
                    [shufflenet_ms_qr_true, 'ShuffleNet(QR)'],
                    [resnet_ms_qr_false, 'ResNet9(CE)'],
                    [resnet_ms_qr_true, 'ResNet9(QR)'],
                    [mlpmixer_ms_qr_false, 'MLPMixer(CE)'],
                    [mlpmixer_ms_qr_true, 'MLPMixer(QR)'],
                    [simplevit_ms_qr_false, 'SimpleViT(CE)'],
                    [simplevit_ms_qr_true, 'SimpleViT(QR)']]


Total files: 4
Total files: 4
Total files: 4
Total files: 4


In [8]:
all_drop_results_ms = []
for result in all_results_ms:
    drop_results = calculate_drop_acc_test(result[0], drop_k=1)
    all_drop_results_ms.append([result[1], drop_results[0], drop_results[1], drop_results[2], drop_results[3], drop_results[4], drop_results[5], drop_results[6], drop_results[7]])
all_drop_results_ms_df = pd.DataFrame(all_drop_results_ms, columns=['model', 'drop_acc', 'increase_loss', 'drop_k', 'base_acc', 'k_acc', 'base_loss', 'k_loss', 'weighted_acc_drop',])
all_drop_results_ms_df

Drop 1 accuracy: 10.147
Drop 1 loss: -6.022
Drop 1 accuracy: -1.015
Drop 1 loss: 11.727
Drop 1 accuracy: -7.297
Drop 1 loss: -16.939
Drop 1 accuracy: -0.345
Drop 1 loss: -15.548
Drop 1 accuracy: 3.893
Drop 1 loss: 2.289
Drop 1 accuracy: 1.140
Drop 1 loss: -10.881
Drop 1 accuracy: -0.181
Drop 1 loss: -13.350
Drop 1 accuracy: 21.891
Drop 1 loss: 26.120


/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupby('exp_num').mean()
/tmp/ipykernel_218973/248924485.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df['exp_type']=='important'].groupb

,model,drop_acc,increase_loss,drop_k,base_acc,k_acc,base_loss,k_loss,weighted_acc_drop
0,ShuffleNet(CE),10.147206,-6.022017,1,0.845814,0.759988,1.087605,1.022109,0.475745
1,ShuffleNet(QR),-1.014698,11.727144,1,0.810094,0.818314,0.938707,1.048790,0.450517
2,ResNet9(CE),-7.297276,-16.939420,1,0.752070,0.806951,0.932356,0.774420,0.007688
3,ResNet9(QR),-0.345463,-15.548182,1,0.787307,0.790027,0.774118,0.653757,0.254397
4,MLPMixer(CE),3.892904,2.289214,1,0.735932,0.707283,0.925499,0.946685,0.165258
5,MLPMixer(QR),1.140431,-10.881358,1,0.726080,0.717800,0.940260,0.837947,0.288982
6,SimpleViT(CE),-0.180630,-13.349605,1,0.702690,0.703959,0.943790,0.817797,0.151357
7,SimpleViT(QR),21.891314,26.119988,1,0.706256,0.551647,0.946512,1.193741,0.530601
